### DOR (Document Ocurrent Representation)
<p><kbd>0226594</kbd> Sara Carolina Gómez Delgado  </p>

In [8]:
import nltk
import matplotlib
from sklearn.ensemble import RandomForestClassifier
import pandas
from bs4 import BeautifulSoup
import numpy as np
import scipy

In [9]:
def get_texts_from_file(path_corpus, path_truth):
    tr_txt = []
    tr_y = []
    with open(path_corpus, "r",encoding="utf8") as f_corpus, open(path_truth, "r",encoding="utf8") as f_truth:
        for tuit in f_corpus:
            tr_txt += [tuit]
        for label in f_truth:
            tr_y += [label] 
    return tr_txt, tr_y

In [10]:
tr_txt, tr_y = get_texts_from_file("./mex_train.txt", "./mex_train_labels.txt")

In [11]:
tr_txt[5].split()

['putos.',
 'no',
 'tienen',
 'madre.',
 'ambriados',
 'mantenidos.',
 'ojetes.',
 'como',
 'es',
 'posible.',
 'mejor',
 'matarlos']

In [12]:
import nltk 
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()

corpus_palabras = []
for doc in tr_txt:
    corpus_palabras += tokenizer.tokenize(doc) # lista con documentos divididos por espacios (contando signos de puntuación)

In [13]:
tr_txt, tr_y = get_texts_from_file("./mex_train.txt", "./mex_train_labels.txt")
val_txt, val_y = get_texts_from_file("./mex_val.txt", "./mex_val_labels.txt")

In [14]:
corpus_palabras = []
for doc in tr_txt:
    corpus_palabras += tokenizer.tokenize(doc) # lista con documentos divididos por espacios (contando signos de puntuación)

In [22]:
vocab_size = 5000
fdist = nltk.FreqDist(corpus_palabras)
vocab = sorted([(fdist[key], key) for key in fdist])[:: -1][: vocab_size]

In [23]:
indices = dict()
for i, w in enumerate(vocab):
    _, word = w
    indices[word] = i
print(len(indices))
list(indices)[:10]

5000


['que', 'de', '.', 'a', 'la', 'y', 'no', 'me', '!', 'el']

In [29]:
def DOR(tweets, V, dict_indices):
    DOR = np.zeros((len(tweets),len(V)), dtype=float)
    for i, tweet in enumerate(tweets):
        word_map = nltk.FreqDist(tokenizer.tokenize(tweet))
        for word in word_map:
            if word in dict_indices:
                dftj = 1 + np.log(word_map[word])
                DOR[i, word_map[word]] = dftj * np.log(len(V)/len(tweet))
    # transformation to document - document
    Dsum = np.zeros( (len(tweets), len(tweets)) )
    for i, c in enumerate(DOR):
        j = np.where(c>0)[0]
        Dsum[i,:] = np.sum(DOR[:,j], axis=1)
    return Dsum


In [32]:
DOR_tr  = DOR(tr_txt, vocab, indices)
DOR_val = DOR(val_txt, vocab, indices)
DOR_tr

array([[10.76021267, 10.30356451, 10.15925425, ...,  4.43965575,
         5.14989736,  4.54690128],
       [10.76021267, 10.30356451, 10.15925425, ...,  4.43965575,
         5.14989736,  4.54690128],
       [10.76021267, 10.30356451, 10.15925425, ...,  4.43965575,
         5.14989736,  4.54690128],
       ...,
       [ 3.99540461,  3.82584531,  3.77226106, ...,  4.43965575,
         5.14989736,  4.54690128],
       [ 3.99540461,  3.82584531,  3.77226106, ...,  4.43965575,
         5.14989736,  4.54690128],
       [ 3.99540461,  3.82584531,  3.77226106, ...,  4.43965575,
         5.14989736,  4.54690128]])